In [1]:
pip install transformers huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# To get model summary
!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import json
import torch
import random
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torchinfo import summary
import re

# Load intents JSON file
with open('intents.json') as file:
    data = json.load(file)

# Extract texts and labels
texts = []
labels = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(intent['tag'])

# Encode the labels
le = LabelEncoder()
labels = le.fit_transform(labels)

In [4]:
# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Set maximum sequence length
max_seq_len = 8

# Tokenize and encode sequences
tokens_train = tokenizer(
    texts,
    max_length=max_seq_len,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Convert to tensors
train_seq = tokens_train['input_ids']
train_mask = tokens_train['attention_mask']
train_labels = torch.tensor(labels)

In [5]:
# Define batch size
batch_size = 16

# Create TensorDataset
train_data = TensorDataset(train_seq, train_mask, train_labels)

# Create DataLoader
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)

In [6]:
# Define the device as CPU
device = torch.device("cpu")

class BERT_Arch(nn.Module):
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, len(le.classes_))
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):
        cls_hs = self.bert(sent_id, attention_mask=mask).last_hidden_state[:, 0]
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# Freeze BERT parameters
for param in model.parameters():
    param.requires_grad = False

# Initialize the model
bert_classifier = BERT_Arch(model)
bert_classifier = bert_classifier.to(device)
summary(bert_classifier)

Layer (type:depth-idx)                                  Param #
BERT_Arch                                               --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              (23,440,896)
│    │    └─Embedding: 3-2                              (393,216)
│    │    └─LayerNorm: 3-3                              (1,536)
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             (42,527,232)
├─Dropout: 1-2                                          --
├─ReLU: 1-3                                             --
├─Linear: 1-4                                           393,728
├─Linear: 1-5                                           131,328
├─Linear: 1-6                                           20,560
├─LogSoftmax: 1-7                                       --
Total

In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define optimizer
optimizer = AdamW(bert_classifier.parameters(), lr=1e-3)

# Compute class weights
class_wts = torch.tensor(compute_class_weight('balanced', classes=np.unique(labels), y=labels), dtype=torch.float).to(device)
cross_entropy = nn.NLLLoss(weight=class_wts)

# Train the model
def train():
    bert_classifier.train()
    total_loss = 0
    total_preds = []

    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch

        preds = bert_classifier(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_classifier.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        preds = preds.detach().numpy()
        total_preds.append(preds)

    avg_loss = total_loss / len(train_dataloader)
    total_preds = np.concatenate(total_preds, axis=0)
    return avg_loss, total_preds

epochs = 200
for epoch in range(epochs):
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    train_loss, _ = train()
    print(f'\nTraining Loss: {train_loss:.3f}')

C:\Users\TIFFANY MUN\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



 Epoch 1 / 200

Training Loss: 4.446

 Epoch 2 / 200

Training Loss: 4.388

 Epoch 3 / 200

Training Loss: 4.377

 Epoch 4 / 200

Training Loss: 4.357

 Epoch 5 / 200

Training Loss: 4.322

 Epoch 6 / 200

Training Loss: 4.276

 Epoch 7 / 200

Training Loss: 4.169

 Epoch 8 / 200

Training Loss: 4.023

 Epoch 9 / 200

Training Loss: 3.885

 Epoch 10 / 200

Training Loss: 3.736

 Epoch 11 / 200

Training Loss: 3.599

 Epoch 12 / 200

Training Loss: 3.491

 Epoch 13 / 200

Training Loss: 3.313

 Epoch 14 / 200

Training Loss: 3.213

 Epoch 15 / 200

Training Loss: 3.054

 Epoch 16 / 200

Training Loss: 2.936

 Epoch 17 / 200

Training Loss: 2.840

 Epoch 18 / 200

Training Loss: 2.732

 Epoch 19 / 200

Training Loss: 2.625

 Epoch 20 / 200

Training Loss: 2.526

 Epoch 21 / 200

Training Loss: 2.343

 Epoch 22 / 200

Training Loss: 2.276

 Epoch 23 / 200

Training Loss: 2.179

 Epoch 24 / 200

Training Loss: 1.949

 Epoch 25 / 200

Training Loss: 1.966

 Epoch 26 / 200

Training Loss: 2

In [8]:
def get_prediction(text):
    text = re.sub(r'[^a-zA-Z ]+', '', text)
    test_text = [text]
    tokens_test_data = tokenizer(
        test_text,
        max_length=max_seq_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    test_seq = tokens_test_data['input_ids']
    test_mask = tokens_test_data['attention_mask']

    bert_classifier.eval()
    with torch.no_grad():
        preds = bert_classifier(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
    preds = np.argmax(preds, axis=1)
    return le.inverse_transform(preds)[0]

def get_response(message):
    intent = get_prediction(message)
    for i in data['intents']:
        if i['tag'] == intent:
            result = random.choice(i['responses'])
            break
    return "Intent: " + intent + '\n' + "Response: " + result

# Example usage
print(get_response("Hello"))

Intent: greeting
Response: Hello there. Tell me how are you feeling today?


In [ ]:
# Conversational loop
def chat():
    print("Start chatting with the bot (type 'quit' to stop)!")
    while True:
        message = input("You: ")
        if message.lower() == 'quit':
            print("Bot: Goodbye!")
            break
        response = get_response(message)
        print(f"Bot: {response}")

# Start the chatbot conversation
chat()

Start chatting with the bot (type 'quit' to stop)!
You: hi
Bot: Intent: greeting
Response: Hello there. Glad to see you're back. What's going on in your world right now?
You: not much
Bot: Intent: neutral-response
Response: Oh I see. Do you want to talk about something?
You: okay
Bot: Intent: casual
Response: Can you elaborate on that?
You: im going through a tough time
Bot: Intent: not-talking
Response: Talking about something really helps. If you're not ready to open up then that's ok. Just know that i'm here for you, whenever you need me.
You: alright
Bot: Intent: casual
Response: Let's discuss further why you're feeling this way.
You: i lost my necklace
Bot: Intent: death
Response: I'm sorry to hear that. If you want to talk about it. I'm here.
You: it was expensive
Bot: Intent: casual
Response: Can you elaborate on that?
You: it was $500
Bot: Intent: neutral-response
Response: Oh I see. Do you want to talk about something?
You: ok bye
Bot: Intent: goodbye
Response: Have a nice day